In [ ]:
import torch
from midi_model import MidiModel

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Running on: {device}")

In [ ]:
mh = MidiModel()
mh.device = device
mh.epochs = 50
mh.batch_size = 32
mh.learning_rate = 0.1

In [ ]:
mh.load_train_data()

In [ ]:
mh.midi.load_files("../data/maestro-v2.0.0/**/*.mid*")

In [ ]:
mh.load_train_notes(0, 10)

In [ ]:
mh.create_model()

In [ ]:
mh.train_model()

In [ ]:
raw_notes = mh.midi.get_notes(mh.midi.files[351])
num_predictions = 50
seq_length = 30
temperature = 0.3

generated_notes, out_pm = mh.generate_notes(raw_notes, num_predictions, seq_length, temperature)

mh.midi.display_midi("output.mid")

In [ ]:
mh.midi.display_audio(out_pm)